In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import matplotlib.pyplot as plt

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11501568/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

In [ ]:
batch_size = 128
epochs = 20

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/20
422/422 [==============================] - 44s 103ms/step - loss: 0.0192 - accuracy: 0.9933 - val_loss: 0.0281 - val_accuracy: 0.9925
Epoch 2/20
422/422 [==============================] - 44s 105ms/step - loss: 0.0183 - accuracy: 0.9940 - val_loss: 0.0278 - val_accuracy: 0.9937
Epoch 3/20
422/422 [==============================] - 44s 103ms/step - loss: 0.0176 - accuracy: 0.9941 - val_loss: 0.0289 - val_accuracy: 0.9930
Epoch 4/20
422/422 [==============================] - 44s 105ms/step - loss: 0.0186 - accuracy: 0.9935 - val_loss: 0.0252 - val_accuracy: 0.9932
Epoch 5/20
422/422 [==============================] - 43s 101ms/step - loss: 0.0166 - accuracy: 0.9943 - val_loss: 0.0277 - val_accuracy: 0.9933
Epoch 6/20
422/422 [==============================] - 43s 101ms/step - loss: 0.0174 - accuracy: 0.9942 - val_loss: 0.0277 - val_accuracy: 0.9932
Epoch 7/20
422/422 [==============================] - 44s 105ms/step - loss: 0.0172 - accuracy: 0.9939 - val_loss: 0.0291 - val_ac

KeyboardInterrupt: ignored

In [ ]:
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image, ImageOps

canvas_html = """
<canvas width=%d height=%d></canvas>
<button>Finish</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')
ctx.lineWidth = %d
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}
var Touch = {x: 0, y: 0}
canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.addEventListener('touchmove', function(e) {
  Touch.x = e.pageX - this.offsetLeft
  Touch.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.ontouchstart = ()=>{
  ctx.beginPath()
  ctx.moveTo(Touch.x, Touch.y)
  canvas.addEventListener('touchmove', onPaint2)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
canvas.ontouchend = ()=>{
  canvas.removeEventListener('touchmove', onPaint2)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}
var onPaint2 = ()=>{
  ctx.lineTo(Touch.x, Touch.y)
  ctx.stroke()
}
var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""

def draw(filename='drawing.png', w=280, h=280, line_width=5):
  display(HTML(canvas_html % (w, h, line_width)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)

  img = Image.open(filename)
  img = img.resize((np.array(img.size)/10).astype(int))
  img = np.array(img)
  img = img[:,:,3]
  plt.imshow(img)
  img = np.reshape(img, (1,28,28,1))
  print(img.shape)
  print(np.argmax(model.predict(img)))

In [ ]:
draw()